# Intro Deep Learning

La idea de este Notebook es mostrar una introducción a modelos de redes neuronales densas, cómo se componen, cómo se entrenan y similitudes con modelos más sencillos como la regresión lineal y logística. Cubriremos conceptos que definen la estructura de estos modelos y los parámetros organizan su aprendizaje.

In [ ]:
!git clone https://github.com/JuanCruzC97/ml-stuff.git

In [ ]:
cd ml-stuff/intro-deep-learning

In [33]:
# Librerías para el manejo de los datos.
import numpy as np
import pandas as pd

# Librerías para visualización.
import plotly.express as px
#import plotly.graph_objects as go

# Funciones propias.
from utils.datasets import make_regression_dataset

# Regresión

En esta primera parte cubrimos un problema muy simple de regresión (predicción de una variable continua). Usaremos una única variable explicativa. Comenzamos generando el dataset que utilizaremos en esta etapa. Se trata de un set de datos sencillo, con una variable explicativa `X` y una variable respuesta `y` continua.

## Dataset



In [34]:
# Generamos los datasets de entrenamiento y evaluación.
train = make_regression_dataset(n_samples=100, noise=1.25, random_state=42)
test = make_regression_dataset(n_samples=30, noise=1.25, random_state=65)

In [35]:
# Vemos algunos valores de la variable explicativa y la respuesta.
train.head()

,X,y
0,-1.003679,-2.201476
1,3.605714,5.682870
2,1.855952,5.638039
3,0.789268,0.695674
4,-2.751851,-1.445551


In [36]:
# También podemos ver algunos estadísticos de ambas variables.
train.describe()

,X,y
count,100.000000,100.000000
mean,-0.238554,1.697763
std,2.379915,3.485939
min,-3.955823,-4.706355
25%,-2.454394,-1.157034
50%,-0.286860,1.241799
75%,1.841625,4.993439
max,3.895095,8.287708


In [37]:
px.scatter(data_frame=train,
           x="X",
           y="y",
           color_discrete_sequence=["#3d5a80"],
           height=500,
           width=800,
           template="plotly_white")

# Modelo Lineal

Por la forma de los datos visualizados podemos darnos cuenta que un modelo lineal simple con la variable `x` tendrá un ajuste pobre, vemos que hay una relación entre la variable `y` y la variable `x` pero esta relación es no es lineal.

$$ y = f(x) + \tilde{\epsilon} \qquad \text{ donde f() es no lineal} $$

El modelo de regresión lineal simple sigue la siguiente expresión, donde $w_0$ y $w_1$ son los parámetros del modelo. Los valores óptimos de estos parámetros debemos encontrarlos, de manera que lleguemos a los valores para los cuales el modelo realice la mejor predicción de la respuesta `y` en función de `x`.

$$\hat{y} = w_0 + w_1 * x$$

¿Cómo conseguimos los mejores valores para los parámetros del modelo? Para eso necesitamos una forma de medir la diferencia entre el valor observado de $y$ para cada $x$ y el valor de predicción del modelo $\hat{y}$ para los $x$ correspondientes. La métrica que usamos para medir esta diferencia es conocida como Suma de Errores Cuadráticos, básicamente, la sumatoria de los residuos elevados al cuadrado para todas las observaciones que tenemos.

$$S = \sum{(y-\hat{y})^2} \implies S = \sum{(y- w_0 + w_1*x)^2} $$

Vemos que la función que mide cuán bueno es el modelo en la predicción tiene como componentes a $y$ y $x$ que son los datos conocidos (observaciones) y también a los parámetros del modelo, por lo que la nuestra función de error depende de los parámetros del modelo $S = f(w_0, w_1)$, por lo que encontrar los valores de los parámetros (también llamados *weights*) óptimos para predecir $\hat{y}$ es un problema de optimización donde 

In [7]:
# Importamos la función del modelo lineal.
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [8]:
# Iniciamos el modelo.
linear_model_sk = LinearRegression()

# Entrenamos el modelo con los datos de train.
linear_model_sk.fit(X=train[["X"]],
                    y=train["y"])

LinearRegression()

In [10]:
print(linear_model_sk.coef_[0])
print(linear_model_sk.intercept_)

1.026409914187006
1.9426174913286312


In [11]:
train["y_preds1"] = linear_model_sk.predict(train[["X"]])
test["y_preds1"] = linear_model_sk.predict(test[["X"]])

print(f'Error Absoluto Promedio {round(mean_absolute_error(train["y"], train["y_preds1"]), 2)}')
print(f'Error Absoluto Promedio {round(mean_absolute_error(test["y"], test["y_preds1"]), 2)}')

Error Absoluto Promedio 2.02
Error Absoluto Promedio 1.84


In [12]:
plot = px.scatter(data_frame=train,
                  x = "X",
                  y = ["y", "y_preds1"],
                  color_discrete_sequence=["#3d5a80", "#ff6700"],
                  height=500,
                  width=800,
                  template ="plotly_white")

#plot.add_traces()

plot.show()

In [27]:
plot = px.scatter(data_frame=train,
                  x = "X",
                  y = train["y"] - train["y_preds1"],
                  #color=color,
                  #color_continuous_scale="viridis", 
                  color_discrete_sequence=["#3d5a80"],
                  height=500,
                  width=800,
                  template ="plotly_white")

#plot.add_traces()

plot.show()

# Regresión Lineal como Neurona

In [19]:
import tensorflow as tf
from tensorflow import keras

from utils.datasets import get_fit_data, get_training_preds

In [20]:
LOSS = "mae"
BATCH_SIZE = 20
EPOCHS = 20
LR = 0.1

In [ ]:
initial_weights = keras.initializers.RandomNormal(0, 0.05, 42)

inputs = keras.Input(shape=(1,))
outputs = keras.layers.Dense(1, kernel_initializer=initial_weights)(inputs)

model1 = keras.Model(inputs, outputs)

model1.compile(optimizer=keras.optimizers.SGD(learning_rate=LR), loss=LOSS)

fit_history = model1.fit(x=train[["X"]], y=train["y"], batch_size=BATCH_SIZE, epochs=EPOCHS, shuffle=True)

In [22]:
print(f"Parámetros del modelo Lineal {[linear_model_sk.coef_[0], linear_model_sk.intercept_]}")

Parámetros del modelo Lineal [1.026409914187006, 1.9426174913286312]


In [24]:
model1.get_weights()

[array([[1.23968]], dtype=float32), array([1.8799994], dtype=float32)]

In [25]:
print(f'Error Absoluto Promedio {round(mean_absolute_error(train["y"], model1.predict(train[["X"]])), 2)}')
print(f'Error Absoluto Promedio {round(mean_absolute_error(test["y"], model1.predict(test[["X"]])), 2)}')

4/4 [==============================] - 0s 4ms/step
Error Absoluto Promedio 1.98
1/1 [==============================] - 0s 61ms/step
Error Absoluto Promedio 1.74


In [53]:
fit_data = get_fit_data(fit_history, 'mae')

px.line(fit_data, 
        x=fit_data.index, 
        y="mae",
        color_discrete_sequence=["#3d5a80"],
        height=500,
        width=800,
        template="plotly_white")

## Entrenamiento



In [40]:
LOSS = "mae"
BATCH_SIZE = 1
EPOCHS = 25
LR = 0.1

initial_weights = keras.initializers.RandomNormal(-0.5, 0.25, 123)

inputs = keras.Input(shape=(1,))
outputs = keras.layers.Dense(1, kernel_initializer=initial_weights)(inputs)

model1 = keras.Model(inputs, outputs)

model1.compile(optimizer=keras.optimizers.SGD(learning_rate=LR), loss=LOSS)

df_weights, df_preds = get_training_preds(train, model1, BATCH_SIZE, EPOCHS)

In [41]:
px.scatter(data_frame=df_preds,
           x="X",
           y=["y", "y_pred"],
           animation_frame="epoch",
           color_discrete_sequence=["#3d5a80", "#ff6700"],
           height=500,
           width=800,
           template="plotly_white")

px.line(df_weights, 
        x=fit_data.index, 
        y="loss",
        color_discrete_sequence=["#3d5a80"],
        height=500,
        width=800,
        template="plotly_white")

## Featuring Engineering

In [ ]:
train["sin_X1"] = np.sin(train["X1"])

linear_model_sk = LinearRegression()

linear_model_sk.fit(X=train[["X1", "sin_X1"]],
                    y=train["y"])

# Old

In [ ]:
px.histogram(train, "X1", nbins=20, height=600, width=600, template="plotly_white")

In [ ]:
px.scatter_3d(data_frame=train,
              x="X1",
              y="X2",
              z="y",
              color="y",
              color_continuous_scale="viridis",
              height=700,
              width=600,
              template="plotly_white")



px.scatter(data_frame=train,
           x = "X2",
           y = "y",
           color="y",
           color_continuous_scale="viridis",
           height=500,
           width=800,
           template ="plotly_white").show()

px.scatter(data_frame=train,
           x = "X1",
           y = "X2",
           color = "y",
           color_continuous_scale="viridis",
           height=500,
           width=800,
           template ="plotly_white").show()